<table style="width: 100%;">
    <tr style="background-color: transparent;"><td>
        <img src="https://d8a-88.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Spring 2020<br>
            Dr. Eric Van Dusen<br>
            Andrei Caprau<br>
        Chris Pyles<br>
        </p></td></tr>
</table>

# Lab 10: Dummy Variables and Multivariable Regression

In [ ]:
from datascience import *
import statsmodels.api as sm
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
plt.style.use("seaborn-muted")
%matplotlib inline
import warnings
warnings.simplefilter("ignore")
import nbforms
feedback = nbforms.Form("feedback_form_config.json")

Welcome to Lab 10! 

In this lab, we'll augment our understanding of linear regression by introducing the concept of **dummy variables**.

Throughout this notebook, we will be predicting an individuals log wage based on different explanatory variables. The dataset comes from the Current Population Survey (CPS), and includes demographic information for a number of individuals, including age, sex, wage, ethnicity, and education level. 

## Dummy Variables

### Motivation

Dummy variables are used to encode categorical data. There are 2 cases when this may occur:

* A variable has non-numeric values, e.g. having `M` or `F` for  `sex`, `HS` or `college` or `Masters` for `education`, and so on.
* A variable has categorical numeric values, e.g. the star ratings of a business on Yelp. 

In the first case, the reason is obvious: we need a way to model a qualitative variable in our model, which is inherently quantitative.

In the second, the reason is more subtle. Even though the data may be quantitative, we would instead like to think of them as belonging in groups or bins; for Yelp stars, there are 3 star restaurants, 3.5 star restaurants, and so on. Typically, the data are not continuous, and doing arithmetic operations such as finding the difference between 2 observations would not too meaningful.

### How Dummy Variables Work

A dummy variable encoding turns the original column of data into many columns, with each new column representing a unique value of the original column. For each observation's value that we seek to encode, the value's corresponding column will be marked with a 1, while all other columns will be marked with a 0. 

For example, if we had a column for sex where `male` and `female` were the only unique values, it would be turned into 2 columns: `male` and `female`. An observation that is male will have the `male` column be 1 and the `female` column be 0.

Let's take a look at dummy variables in action, by first reading in the CPS dataset.

In [ ]:
cps = Table.read_table("cps.csv")
cps.sample()

Notice that race is already dummy encoded for us: instead of having a column called `race` with values like `hispanic`, `black`, `asian`, or `white`, we instead have 3 columns representing the 4 different races:
- If an individual is hispanic, then their value for the `hispanic` column will be 1, and value for `black` and `asian` will be 0.
- If an individual is black, then their value for the `black` column will be 1, and value for `hispanic` and `asian` will be 0.
- If an individual is asian, then their value for the `asian` column will be 1, and value for `hispanic` and `black` will be 0.
- If an individual is white, then they are neither `hispanic`, `black`, or `asian`. Thus, all 3 columns will be 0. 

*Note*: we also could have incorporated `white` as a column as well, but 3 columns is sufficient to depict all 4 categories. In practice, we typically drop one column for dummy encoding; there is a more complicated reason for why we do this, but we won't dive into it here. 

What other variables are already dummy encoded?

### Creating Dummy Variables

Let's start by creating dummy variables for 200 flips of a coin. For each flip, we have two possible values: `H` or `T`.

In [ ]:
flips = Table().with_column("flip", np.random.choice(make_array("H", "T"), 200))
flips

To create dummy variables, we need to know all of the **unique** values of our variable, since we will need to create one new column for each of these. Although we already know our variable has only 2 possible values, "H" and "T", let's practice anyway. The function `np.unique` will give you an array of the unique values of the array passed to it.

**Question 1.1:** Assign `unique_vals` to an array of the unique values in the `flip` column of `flips`.

In [ ]:
unique_vals = np.unique(...)
unique_vals

In [ ]:
## Solution ##
unique_vals = np.unique(flips.column("flip"))
unique_vals

Now that we know these values, we want to create a column for each value with a 1 if the value for that row equals the column value, and a 0 otherwise. To do this, we'll begin by creating dummy encodings with `True` and `False` values, instead of 1's and 0's. 

**Question 1.2:** For each value in `unique_vals`, add a column to `flips` with `True` or `False` values, instead of 1's and 0's. For example, if a flip was heads, then `flip_H` will be `True` and `flip_T` will be `False`.

*Hint*: you can check whether a column is equal to a certain value with `==`. 

In [ ]:
for val in unique_vals:
    dummy_vals = ...
    flips = flips.with_column("flip_" + val, ...)

flips

In [ ]:
## Solution ##
for val in unique_vals:
    dummy_vals = flips.column('flip') == val
    flips = flips.with_column("flip_" + val, dummy_vals)

flips

Notice that we've created columns with names of the format `flips_{value}`. However, we still have a problem: these columns have boolean values, not integers!

**Question 1.3:** Cast our boolean values in `flip_H` and `flip_T` to integers using `Table.apply`. Recall that we can **typecast** a boolean to an integer by calling the `int` function on it. This will map `True` to 1 and `False` to 0. You can pair this with `Table.apply`.

In [ ]:
for val in unique_vals:
    int_vals = flips.apply(..., ...)
    flips = flips.with_column("flip_" + val, ...)
    
flips

In [ ]:
## Solution ##
for val in unique_vals:
    int_vals = flips.apply(int, "flip_" + val)
    flips = flips.with_column("flip_" + val, int_vals)
    
flips

Now we're almost there! The last thing we want to do is get rid of the original column, so that it doesn't muck up any analysis we do later on. Let's drop it with `Table.drop`.

**Question 1.4:** Create a new table `flips_dummy` that drops the original `flip` column from `flips`.

In [ ]:
flips_dummy = ...
flips_dummy

In [ ]:
## Solution ##
flips_dummy = flips.drop("flip")
flips_dummy

Congratulations, you've now created dummy variables for a categorical variable! Notice that our choice to iterate through the unique values means that we can use this same logic for any arbitrarily large number of unique values. The function `get_dummies` defined below encapsulates this logic that we've built, albeit with a simplified encoding step. Note that this function also drops the first of the unique values by default via the `drop_first` argument. This function will be provided for you in the project.

In [ ]:
def get_dummies(tbl, col, drop=True, drop_first=True):
    """Creates dummy variables for a column of a table"""
    values = np.unique(tbl.column(col))
    if drop_first:
        values = values[1:]
    for val in values:
        encoding = tbl.apply(lambda s: int(s == val), col)
        tbl = tbl.with_column(col + "_" + str(val), encoding)
    if drop:
        tbl = tbl.drop(col)
    return tbl

For example, we can call it on the original flips table.

In [ ]:
get_dummies(flips.select('flip'), 'flip', drop_first = False)

By default, `get_dummies` will drop the original column and one of the dummy columns. In our example, `flip` and `flip_H` will be dropped, but that's ok: we can still figure out whether a flip is head or tails by looking at `flip_T` alone. If `flip_T` is 1, then the flip came up tails; if `flip_T`, is 0, then the flip came up heads. This same logic can be applied to columns with more than 2 different unique values.

In [ ]:
get_dummies(flips.select('flip'), 'flip')

### Dummy Variables for CPS

Now that we have a way to get dummy variables, let's apply this to our CPS data. We want to get dummies for the `educ` variable, which indicates the number of years of schooling of an individual.

**Question 1.5:** Apply our `get_dummies` function to `cps` on the `educ` column.

In [ ]:
cps = get_dummies(..., ...)
cps

In [ ]:
## Solution ##
cps = get_dummies(cps, "educ")
cps

## Multivariable Regression

We are now ready to perform some multivariable regression. Recall that in this form, our regression line has the formula

$$\Large
\hat{y} = \hat{\alpha} + \hat{\beta}_1 x_1 + \cdots + \hat{\beta}_n x_n
$$

where we predict $y$ with $n$ features.

To do multivariable regression in Python, we use the `statsmodels` package, which has a very simple flow:

```python
X = tbl.select(features).values             # Separate features (explanatory and control variables) 
Y = tbl.column(target)                      # Separate outcome variable
model = sm.OLS(Y, sm.add_constant(X))       # Initialize the OLS regression model
result = model.fit()                        # Fit the regression model and save it to a variable
result.summary()                            # Display a summary of results
```

Let's start by demonstrating a simple version of using `statsmodels` by performing single variable regression. We start by selecting out the column we'll be regressing on and the outcome variable. Note that we take out the values of each by adding `.values` to the end. This is because `statsmodels` only knows how to work with NumPy arrays, not tables.

In [ ]:
X = cps.select("age").values          # Regressing on age
Y = cps.column("logwage")             # Outcome is log(wage)

Now that we have our data, we can create an `sm.OLS` object to create out model and fit it. Note that when we call `sm.OLS`, we pass `Y` and `X` as arguments, but we wrap `X` in a call to `sm.add_constant`. This has the effect of adding a column of ones to the data, so that our output includes a value for the intercept.

In [ ]:
model = sm.OLS(Y, sm.add_constant(X))

Now that we have a model, we need to fit it. We capture the result of fitting in a new variable `result`.

In [ ]:
result = model.fit()

Finally, let's look at our coefficients by calling `result.summary()`.

In [ ]:
result.summary()

Now that we have demonstrated `statsmodels` with the single variable case, extension to the multivariable case is straightforward.

**Question 2.1:** Complete the model below that regresses `logwage` on all of our `educ` dummies. Display the results summary. We have provided the labels of all the `educ` dummy variables for you in `educ_cols`.

In [ ]:
educ_cols = make_array('educ_3', 'educ_6', 'educ_8', 'educ_9', 'educ_10', 'educ_11', 'educ_12', 'educ_13', 
                       'educ_14', 'educ_16', 'educ_18', 'educ_20')

X = cps.select(...).values
Y = cps.column(...)

model = sm.OLS(..., ...)
result = ...
result.summary()

In [ ]:
## Solution ##
educ_cols = make_array('educ_3', 'educ_6', 'educ_8', 'educ_9', 'educ_10', 'educ_11', 'educ_12', 'educ_13', 
                       'educ_14', 'educ_16', 'educ_18', 'educ_20')

X = cps.select(educ_cols).values
Y = cps.column("logwage")

model = sm.OLS(Y, sm.add_constant(X))
result = model.fit()
result.summary()

**Question 2.2:** Interpret the regression coefficients. What is the benefit of an undergraduate degree (`educ_16`, $\beta_{10}$) compared to a high school diploma (`educ_12`, $\beta_7$).

_Type your answer here, replacing this text._

<div class="alert alert-danger">
    
<strong>SOLUTION:</strong> We see that $\beta_7$ is 0.3231 and $\beta_{10}$ is 0.8291, meaning that we gain a benefit of $0.8291 - 0.3231 = 0.506$ in log wages.
    
</div>

**Question 2.3:** Create your own regression in the cell below.

In [ ]:
X = ...
Y = ...

model = ...
result = ...
result.summary()

---

### Feedback

Please fill out the feedback form below regarding this lecture.

In [ ]:
feedback.ask()